 * @ Author: Yohei Ohto
 * @ Create Time: 2024-11-25 22:24:43
 * @ Modified time: 2024-11-25 22:24:47
 * @ Description: sqlite3のコマンドの解説

## レベルとしては基本情報技術者試験に少し追加事項があるくらい
## このqiitaも参考になる https://qiita.com/meshi0323/items/ecbbd57e54fe83308ef9
## https://qiita.com/studio_haneya/items/2626a11bb69155872090

# install sqlite3

In [1]:
!pip install pysqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.4-cp310-cp310-linux_x86_64.whl size=154390 sha256=ca968d134531ee943e37e0189c9080dcd6fcf174864645412bc713d6df85610c
  Stored in directory: /root/.cache/pip/wheels/04/55/74/e3b53aeb9ff9e933626ee3938ccad69123828115ae34ce028e
Successfully built pysqlite3

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import sqlite3

## VS CODEで使用する場合には SQLite Viewer を入れると、データベースの中身を確認できるためオススメ

# databaseの作成 (connect)

In [4]:
dbname = 'database/demo.db'
conn = sqlite3.connect(dbname)

# カーソルの作成
データベースと対話、管理するためのインターフェイス  
SQLクエリを実行したり、その結果を操作したりするために使用

In [ ]:
cursor = conn.cursor()
# cur = conn.cursor() という宣言文も多い

# TABLEの作成
tableを作成する  
イメージとしてはエクセルのページ1枚を作成する感じ  
ただし、カラム名を設定する必要があり、そこに制約をかけることができる。

In [27]:
# 簡単にはこれでOK
cursor.execute("""
CREATE TABLE orders (
    order_id,
    user_id,
    order_date,
    total_amount,
    status
)
""")

In [21]:
# 制限を盛りに盛るとこんな感じ
cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    total_amount REAL NOT NULL CHECK(total_amount > 0),
    status TEXT NOT NULL DEFAULT 'pending',
    FOREIGN KEY (user_id) REFERENCES users(id),
    UNIQUE (order_id, user_id)
)
""")

IF NOT EXISTS __ __というテーブルが存在していれば
  
INTEGER 整数値 int  
REAL 浮動小数点 float  
TEXT テキスト型	str  
BLOB バイナリ型	bytes  
NUMERIC	数値型	int/float  
  
PRIMARY KEY 主キー (重複が許されない、かつ欠損値が許されない → これを使えば検索が一意にできる)  
AUTOINCREMENT 前の値+1の値が自動で追加される  
NOT NULL 欠損値が許されない  
UNIQUE 重複は許されない  
DEFAULT デフォルト値を設定する  
CHECK ()内の条件を満たすようにする  
FOREIGN KEY 他のテーブルのキーを参照する (ここでは、usersというテーブルのidを参照する)  

# TABLEの削除

In [12]:
cursor.execute("""
DROP TABLE IF EXISTS orders 
""")

テーブルを削除すると外部キーの参照を外す必要がある

In [ ]:
cursor.execute("""
PRAGMA foreign_keys = OFF;
""")

cursor.execute("""
DROP TABLE orders;
""")

cursor.execute("""
PRAGMA foreign_keys = ON
""")

# TABLEの操作

In [ ]:
# テーブル名を変更する
cursor.execute("""
ALTER TABLE orders RENAME TO new_orders
""")

In [ ]:
# テーブルに新しい列を追加 descriptionというstr型のデータを格納する列が追加されている
cursor.execute("""
ALTER TABLE orders ADD COLUMN description TEXT
""")

In [ ]:
# sqliteでは列の削除をDROPでできない
# そのため、以下のように必要なデータのみ持つコピーの作成をしてから削除する

# 1. 必要な列だけを持つ新しいテーブルを作成
cursor.execute("""
CREATE TABLE orders_new (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    total_amount REAL NOT NULL
)
""")

# 2. データをコピー
cursor.execute("""
INSERT INTO orders_new (order_id, user_id, order_date, total_amount)
SELECT order_id, user_id, order_date, total_amount
FROM orders
""")

# 3. 元のテーブルを削除
cursor.execute("""
DROP TABLE orders
""")

# 4. 新しいテーブルを元の名前にリネーム
cursor.execute("""
ALTER TABLE orders_new RENAME TO orders
""")


# データの操作 (INSERT文)

In [ ]:
# executeでデータを挿入 必ずタプルで入れる
# 基本的に”””で囲われている部分の？が、その外部の指定で参照の対象となる
cursor.execute("""
INSERT INTO orders (order_id, user_id, order_date, total_amount, status) VALUES (?, ?, ?, ?, ?)""", 
(1, 1, 241126, 2, 'pending')
)

In [30]:
# execute manyでデータを複数挿入 必ずタプルのリストで入れる
cursor.executemany("""
INSERT INTO orders (order_id, user_id, order_date, total_amount, status) VALUES (?, ?, ?, ?, ?)""",
[(1, 1, 241126, 2, 'pending'), (2, 1, 241126, 4, 'pending')])

# データの操作 (UPDATE文)

In [ ]:
# order_idが1のデータのstatusを"completed"に
cursor.execute("""
UPDATE orders SET status = ? WHERE order_id = ?
""", ("completed", 1))

# データの抽出 (SELECT文)

In [ ]:
# 特定のカラム(order_id, user_id)を取得
cursor.execute("""
SELECT order_id, user_id FROM orders
""")
rows = cursor.fetchall()

# 全てのカラムのデータを取得
# *が全てのカラムを参照する
cursor.execute("""
SELECT * FROM orders
""")
rows = cursor.fetchall()

In [ ]:
# 条件付きで特定のカラム(order_id, user_id)を取得 WHERE句を使用
cursor.execute("""
SELECT order_id, user_id FROM orders WHERE total_amount = ?
""", 
(200,))
rows = cursor.fetchall()

# total_amount の後ろには=以外にも、>, >=, <, <=が指定できる

In [ ]:
# 複数の条件を指定してデータを取得 WHERE句で条件と論理演算子を使用
cursor.execute("""
SELECT * FROM orders WHERE total_amount > ? AND status = ?
""", 
(100, 'pending'))
rows = cursor.fetchall()  # データを全件取得

# AND, ORの論理演算子が使用できる

# 範囲条件を指定してデータを取得 WHERE句でBETWEENを使用
cursor.execute("""
SELECT * FROM orders WHERE total_amount BETWEEN ? AND ?
""", 
(50, 200))
rows = cursor.fetchall() 

# 最大値、最小値間で含まれる場合(最大値、最小値を含む)、BETWEEN A AND Bが使用できる

# 特定の値に一致するデータを取得 WHERE句でINを使用
cursor.execute("""
SELECT * FROM orders WHERE status IN (?, ?)
""", 
('pending', 'completed'))
rows = cursor.fetchall()

In [ ]:
# 任意の複数文字を含むパターンを検索 WHERE句でLIKEと%を使用 → ingという文字列が入っている情報を抜き出すことができる
cursor.execute("""
SELECT * FROM orders WHERE status LIKE ?
""", 
('%ing',))
rows = cursor.fetchall()

# 任意の1文字を含むパターンを検索 WHERE句でLIKEと_を使用 →　pendingのようにendingの前に一文字ある場合に使用できる
cursor.execute("""
SELECT * FROM orders WHERE status LIKE ?
""", 
('_ending',))
rows = cursor.fetchall()

# 論理演算子が使用できる
# 'pen'を含むが、5文字以上のstatusを検索
cursor.execute("""
SELECT * FROM orders WHERE status LIKE ? AND LENGTH(status) > ?
""", 
('%pen%', 5))

In [ ]:
# 指定した列で昇順にデータを並べ替え WHERE句で条件を使用　デフォルトなのでASCは省略可能
cursor.execute("""
SELECT * FROM orders ORDER BY total_amount ASC
""")
rows = cursor.fetchall()

# 指定した列で降順にデータを並べ替え
cursor.execute("""
SELECT * FROM orders ORDER BY order_date DESC
""")
rows = cursor.fetchall()

# 複数の列を指定してデータを並べ替え　まずstatus列で昇順に並べ、次にtotal_amount列で降順に並べる
cursor.execute("""
SELECT * FROM orders ORDER BY status ASC, total_amount DESC
""")
rows = cursor.fetchall()

In [ ]:
# 条件を指定して合計値を計算
cursor.execute("""
SELECT SUM(total_amount) FROM orders WHERE status = ?
""", 
('completed',))  # 'completed' 状態の注文だけ合計
result = cursor.fetchone()

# 合計 SUM() 
# 平均 AVG()
# 最大 MAX()
# 最小 MIN()　が使用dふぇきる

In [ ]:
# 指定した列のデータ件数をカウント
cursor.execute("""
SELECT COUNT(total_amount) FROM orders
""")
result = cursor.fetchone()  

In [ ]:
# 'status'列でグループ化
# ユニークなステータスの値が取ってこれる
cursor.execute("""
SELECT status FROM orders GROUP BY status
""")
rows = cursor.fetchall() 

# 'status'列でグループ化し、各グループの行数をカウント
# ユニークなステータスの値とその数が取ってこれる
cursor.execute("""
SELECT status, COUNT(*) FROM orders GROUP BY status
""")
rows = cursor.fetchall() 

# 'user_id' と 'status' 列でグループ化し、各組み合わせの注文数をカウント
cursor.execute("""
SELECT user_id, status, COUNT(*) FROM orders GROUP BY user_id, status
""")
rows = cursor.fetchall()

# このように複数のデータを取ってきたい場合にはSELECT後に,でつなぐ
# AVE()なども使用できる

In [ ]:
# 列に別名を付けてデータを取得
cursor.execute("""
SELECT order_id AS num FROM orders
""")
rows = cursor.fetchall() 

# 複数列に別名を付けてデータを取得
cursor.execute("""
SELECT order_id AS num, total_amount AS amount FROM orders
""")
rows = cursor.fetchall()  # 結果を全件取得

In [ ]:
# 'status' 列を集計し、注文数が5件を超えるステータスを取得
cursor.execute("""
SELECT status FROM orders GROUP BY status HAVING COUNT(*) > 5
""")
rows = cursor.fetchall()

# 'status' 列でグループ化し、注文数が5件を超えるステータスを表示
cursor.execute("""
SELECT status, COUNT(*) FROM orders GROUP BY status HAVING COUNT(*) > 5
""")
rows = cursor.fetchall()

# 'total_amount' が 100 より大きい注文を対象に、'status' 列でグループ化し、合計金額が 500 を超えるステータスを表示
cursor.execute("""
SELECT status, SUM(total_amount) FROM orders WHERE total_amount > 100 GROUP BY status HAVING SUM(total_amount) > 500
""")

# commit (超重要)

デフォルトで自動的にcommitしてくれるが、明示することでその操作による変更を確実に反映させる。  
cursor.execute("BEGIN TRANSACTION;")と指定することにより手動commitモードになる。  
手動commitモードでは、そのトランザクション（操作）によってエラーが生じた場合には、rollback()によって変更を取り消すことができる。  

In [ ]:
cursor.execute("BEGIN TRANSACTION;")

# connect時を確認。cursorではない
connection.commit()
# エラーが発生した場合、ロールバック
connection.rollback()

# fetchについて

In [ ]:
# fetchoneはタプルで結果を一つ返す
# 該当しているものがなければ、Noneを返す
result = cursor.fetchone()

# 1つのcursorに二つ指定すると、複数ある場合には、順番に表示される
# 該当しているものがなければ、Noneを返す
result_1 = cursor.fetchone()
result_2 = cursor.fetchone()

# タプルのリストの形でcursorでヒットしたすべてのデータを返す。
result = cursor.fetchall()

# 引数に整数nを指定することで、n個ずつ結果を表示する
# fetchoneの複数個版
# defaultはn=1で、複数取る場合にはタプルのリストになる。
result = cursor.fetchmany(2)

# PRAGMA関数
sqlite3特有の関数  
データベースそのものの挙動や設定を司る  
基本的に使用する可能性があるのは、外部キーとの制約(前述)、整合性の確認、くらい？  

In [ ]:
# 整合性の確認
cursor.execute("PRAGMA integrity_check;")

# ユーザの作成、権限管理
→ 省略